# PARCIAL 2
## MAKSIM NAGI


In [1]:
import sqlite3
from datetime import datetime
from dataclasses import dataclass
import os
from pathlib import Path

# --- CONFIGURACIÓN DE RUTA DE LA BASE DE DATOS ---

# Se crea (si no existe) una carpeta dentro del home del usuario para guardar la base de datos
BASE_DIR = Path.home() / "SabanaEats_DB"
BASE_DIR.mkdir(parents=True, exist_ok=True)

# Ruta completa del archivo SQLite
DB_PATH = BASE_DIR / "sabana_eats.db"

print(f"Base de datos se guardará en: {DB_PATH}")

# --- FUNCIONES DE CONEXIÓN Y CREACIÓN DE TABLAS ---

def get_conn():
    """Retorna una conexión activa a la base de datos."""
    return sqlite3.connect(DB_PATH)

def init_db():
    """
    Crea las tablas principales si no existen:
    - menu: catálogo de productos
    - pedidos: encabezado de cada orden
    - detalles_pedido: relación entre pedidos y productos
    """
    with get_conn() as conn:
        cur = conn.cursor()

        # Tabla de productos disponibles
        cur.execute("""
        CREATE TABLE IF NOT EXISTS menu (
            id INTEGER PRIMARY KEY,
            nombre_producto TEXT NOT NULL,
            precio_base REAL NOT NULL,
            tipo TEXT NOT NULL CHECK(tipo IN ('comida','bebida','postre'))
        )
        """)

        # Tabla de encabezado de pedidos
        cur.execute("""
        CREATE TABLE IF NOT EXISTS pedidos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre_cliente TEXT NOT NULL,
            fecha_hora TEXT NOT NULL,
            total_pagado REAL NOT NULL
        )
        """)

        # Tabla de detalle: productos dentro de cada pedido
        cur.execute("""
        CREATE TABLE IF NOT EXISTS detalles_pedido (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pedido_id INTEGER NOT NULL,
            producto_id INTEGER NOT NULL,
            cantidad INTEGER NOT NULL CHECK(cantidad > 0),
            FOREIGN KEY (pedido_id) REFERENCES pedidos(id),
            FOREIGN KEY (producto_id) REFERENCES menu(id)
        )
        """)

        conn.commit()

def seed_menu_if_empty():
    """
    Si la tabla 'menu' está vacía, inserta productos iniciales.
    Esto asegura que haya datos disponibles para probar el sistema.
    """
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute("SELECT COUNT(*) FROM menu")
        count = cur.fetchone()[0]

        if count == 0:
            productos_iniciales = [
                (1, "Hamburguesa", 15000.0, "comida"),
                (2, "Ensalada", 12000.0, "comida"),
                (3, "Gaseosa", 5000.0, "bebida"),
                (4, "Jugo Natural", 6000.0, "bebida"),
                (5, "Brownie", 7000.0, "postre"),
                (6, "Helado", 8000.0, "postre"),
            ]
            cur.executemany(
                "INSERT INTO menu (id, nombre_producto, precio_base, tipo) VALUES (?, ?, ?, ?)",
                productos_iniciales
            )
            conn.commit()

# Inicializa la base de datos y carga datos si es la primera ejecución
init_db()
seed_menu_if_empty()
print("Base de datos lista y menú disponible.")

# --- MODELO POO (PRODUCTO + HERENCIA) ---

@dataclass
class Producto:
    """
    Clase base que representa un producto genérico del menú.
    Se usa 'dataclass' para reducir código repetitivo (constructor, repr, etc.)
    """
    id: int
    nombre: str
    precio_base: float

    def calcular_impuesto(self) -> float:
        """
        Método polimórfico que será redefinido por las subclases.
        """
        raise NotImplementedError

# Subclases que aplican diferentes reglas de impuesto según el tipo de producto

@dataclass
class Comida(Producto):
    def calcular_impuesto(self) -> float:
        return round(self.precio_base * 0.08, 2)  # 8% de impuesto

@dataclass
class Bebida(Producto):
    def calcular_impuesto(self) -> float:
        return round(self.precio_base * 0.12, 2)  # 12% de impuesto

@dataclass
class Postre(Producto):
    def calcular_impuesto(self) -> float:
        return 0.0  # Los postres están exentos

# --- FUNCIONES PARA CARGAR Y MOSTRAR DATOS DEL MENÚ ---

from typing import Dict, Tuple, List

def fetch_menu_rows() -> List[Tuple]:
    """Devuelve todas las filas del menú directamente desde la base de datos."""
    with get_conn() as conn:
        cur = conn.cursor()
        cur.execute("SELECT id, nombre_producto, precio_base, tipo FROM menu ORDER BY id ASC")
        return cur.fetchall()

def construir_objeto_producto(row: Tuple) -> Producto:
    """
    Convierte una fila del menú (tupla) en una instancia de la clase correspondiente.
    Aplica polimorfismo al crear objetos Comida/Bebida/Postre según el tipo.
    """
    pid, nombre, precio, tipo = row
    if tipo == "comida":
        return Comida(pid, nombre, float(precio))
    elif tipo == "bebida":
        return Bebida(pid, nombre, float(precio))
    elif tipo == "postre":
        return Postre(pid, nombre, float(precio))
    else:
        raise ValueError(f"Tipo desconocido: {tipo}")

def cargar_menu_objetos() -> Dict[int, Producto]:
    """
    Carga el menú completo desde la base y lo transforma en un diccionario
    de objetos {id: Producto}.
    """
    objetos = {}
    for row in fetch_menu_rows():
        obj = construir_objeto_producto(row)
        objetos[obj.id] = obj
    return objetos

def mostrar_menu():
    """Muestra el menú en formato tabular simple."""
    rows = fetch_menu_rows()
    print("\n--- MENÚ DE PRODUCTOS ---")
    print("ID | Nombre           | Precio Base | Tipo")
    print("---------------------------------------------")
    for pid, nombre, precio, tipo in rows:
        print(f"{pid:<2} | {nombre:<15} | ${precio:>10.2f} | {tipo}")

# --- LÓGICA DE NEGOCIO: CREAR PEDIDOS ---

def input_non_empty(prompt: str) -> str:
    """Solicita texto no vacío (valida que el usuario no deje el input en blanco)."""
    while True:
        s = input(prompt).strip()
        if s:
            return s
        print("Entrada vacía. Intente de nuevo.")

def input_int(prompt: str, min_value: int = None) -> int:
    """Solicita un entero válido, opcionalmente mayor o igual que un valor mínimo."""
    while True:
        try:
            val = int(input(prompt).strip())
            if min_value is not None and val < min_value:
                print(f"El valor debe ser >= {min_value}.")
                continue
            return val
        except ValueError:
            print("Ingrese un número entero válido.")

def crear_pedido():
    """Gestiona la creación completa de un pedido, desde la entrada hasta el guardado."""
    nombre_cliente = input_non_empty("Ingrese su nombre: ")

    # Carga el menú actual como objetos POO
    menu_obj = cargar_menu_objetos()
    if not menu_obj:
        print("El menú está vacío. No se puede crear el pedido.")
        return

    mostrar_menu()
    print("\nAgregue productos por ID. Escriba 'fin' para terminar.")

    items: List[Tuple[int, int]] = []  # Lista de (producto_id, cantidad)

    # Bucle de selección de productos
    while True:
        id_raw = input("ID de producto (o 'fin'): ").strip().lower()
        if id_raw == "fin":
            break
        if not id_raw.isdigit():
            print("Debe ingresar un ID numérico o 'fin'.")
            continue

        pid = int(id_raw)
        if pid not in menu_obj:
            print("ID no encontrado en el menú.")
            continue

        cantidad = input_int("Cantidad: ", min_value=1)
        items.append((pid, cantidad))
        print("Añadido.")

    if not items:
        print("No se agregaron productos. Pedido cancelado.")
        return

    # Calcula el total (precio base + impuesto) * cantidad
    total = 0.0
    print("\n--- RESUMEN DEL PEDIDO ---")
    for pid, cantidad in items:
        p: Producto = menu_obj[pid]
        impuesto = p.calcular_impuesto()
        precio_unit_final = round(p.precio_base + impuesto, 2)
        subtotal = round(precio_unit_final * cantidad, 2)
        total += subtotal
        print(f"{p.nombre} x{cantidad} -> unit: ${precio_unit_final:.2f} (incl. imp ${impuesto:.2f}) | sub: ${subtotal:.2f}")

    total = round(total, 2)
    print(f"TOTAL A PAGAR: ${total:.2f}")

    # Inserta el pedido en la base de datos
    fecha_hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with get_conn() as conn:
        cur = conn.cursor()
        # Encabezado del pedido
        cur.execute(
            "INSERT INTO pedidos (nombre_cliente, fecha_hora, total_pagado) VALUES (?, ?, ?)",
            (nombre_cliente, fecha_hora, total)
        )
        pedido_id = cur.lastrowid

        # Detalle del pedido
        cur.executemany(
            "INSERT INTO detalles_pedido (pedido_id, producto_id, cantidad) VALUES (?, ?, ?)",
            [(pedido_id, pid, cant) for pid, cant in items]
        )
        conn.commit()

    print(f"\n✅ Pedido #{pedido_id} guardado correctamente en la base de datos.")

# --- INTERFAZ DE USUARIO: MENÚ INTERACTIVO ---

def menu_interactivo():
    """Menú principal del programa. Permite navegar entre las opciones."""
    while True:
        print("\n=== SabanaEats ===")
        print("1. Ver menú de productos")
        print("2. Crear nuevo pedido")
        print("3. Salir")

        opcion = input("Seleccione una opción: ").strip()

        if opcion == "1":
            mostrar_menu()

        elif opcion == "2":
            try:
                crear_pedido()
            except Exception as e:
                # Captura general para evitar que el programa se detenga por errores no previstos
                print(f"❗ Ocurrió un error creando el pedido: {e}")

        elif opcion == "3":
            print("Gracias por usar SabanaEats. ¡Hasta luego!")
            break

        else:
            print("Opción inválida. Intente de nuevo.")

# --- EJECUCIÓN DEL PROGRAMA ---

menu_interactivo()


Base de datos se guardará en: /Users/maksimnagi/SabanaEats_DB/sabana_eats.db
Base de datos lista y menú disponible.

=== SabanaEats ===
1. Ver menú de productos
2. Crear nuevo pedido
3. Salir

--- MENÚ DE PRODUCTOS ---
ID | Nombre           | Precio Base | Tipo
---------------------------------------------
1  | Hamburguesa     | $  15000.00 | comida
2  | Ensalada        | $  12000.00 | comida
3  | Gaseosa         | $   5000.00 | bebida
4  | Jugo Natural    | $   6000.00 | bebida
5  | Brownie         | $   7000.00 | postre
6  | Helado          | $   8000.00 | postre

Agregue productos por ID. Escriba 'fin' para terminar.
Añadido.
Añadido.

--- RESUMEN DEL PEDIDO ---
Brownie x1 -> unit: $7000.00 (incl. imp $0.00) | sub: $7000.00
Helado x1 -> unit: $8000.00 (incl. imp $0.00) | sub: $8000.00
TOTAL A PAGAR: $15000.00

✅ Pedido #4 guardado correctamente en la base de datos.

=== SabanaEats ===
1. Ver menú de productos
2. Crear nuevo pedido
3. Salir
Gracias por usar SabanaEats. ¡Hasta luego!
